In [1]:
import langchain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI



In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key=os.getenv("openai_api_key")

In [3]:
client=ChatOpenAI(api_key=api_key, model_name="gpt-3.5-turbo", temperature=0.5)

/home/saugat/Desktop/GenAI/GenAI_env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [4]:
RESPONSE_JSON={
    '1':{
        'mcq':'multiple choice questions',
        'options':{
            'a':'choice here',
            'b':'choice here',
            'c':'choice here',
            'c':'choice here',
            'd':''
        },
        'correct':'correct answer',
    },

    '3':{
        'mcq':'multiple choice questions',
        'options':{
            'a':'choice here',
            'b':'choice here',
            'c':'choice here',
            'c':'choice here',
            'd':''
        },
        'correct':'correct answer',
    },

    '3':{
        'mcq':'multiple choice questions',
        'options':{
            'a':'choice here',
            'b':'choice here',
            'c':'choice here',
            'c':'choice here',
            'd':''
        },
        'correct':'correct answer',
    },
    
}

In [5]:
Template1="""
text:{text}
you are a mcq generator.Based on the given text, your job is to generate {number} multiple choice question for
{subject} student in {tone} tone.Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure the format is in RESPONSE_JSON format which is given below and use it as a guide.
Enure to make {number} MCQs

{response_json}

"""

In [7]:
quiz_generation_prompt=PromptTemplate(
    input_variables=['text','number','subject','tone','response_json'],
    template=Template1
)

In [8]:
from langchain.chains import LLMChain
quiz_creation_chain=LLMChain(llm=client, prompt=quiz_generation_prompt,output_key='quiz',verbose=True)

/home/saugat/Desktop/GenAI/GenAI_env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [9]:


Template2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""



In [10]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=['subject','quiz'],
    template=Template2
)

In [11]:
from langchain.chains import LLMChain
quiz_evaluation_chain=LLMChain(llm=client, prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [12]:
from langchain.chains import SequentialChain

In [13]:
final_chain=SequentialChain(
    chains=[quiz_creation_chain,quiz_evaluation_chain],
    input_variables=['text','number','subject','tone','response_json'],
    output_variables=['quiz','review'],
    verbose=True
                            )



In [14]:
file_path="/home/saugat/Desktop/GenAI/data.txt"

In [15]:
with open(file_path,"r") as f:
    text=f.read()

In [16]:
text

'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, 

In [17]:
RESPONSE_JSON

{'1': {'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': ''},
  'correct': 'correct answer'},
 '3': {'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': ''},
  'correct': 'correct answer'}}

In [19]:
number=5
subject="biology"
tone='simple'

In [20]:
from langchain.callbacks import get_openai_callback
import json

with get_openai_callback() as cb:
    response=final_chain(
        {
            'text':text,
            'number':number,
            "subject":subject,
            'tone':tone,
            "response_json":json.dumps(RESPONSE_JSON)
        }
    )

/home/saugat/Desktop/GenAI/GenAI_env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [21]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1422
Prompt Tokens:1001
Completion Tokens:421
Total Cost:0.0023435


In [22]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [23]:
quiz=response.get('quiz')

In [24]:
quiz

'{\n"1": {\n"mcq": "What is biology?",\n"options": {\n"a": "The study of rocks",\n"b": "The scientific study of life",\n"c": "The study of outer space",\n"d": "The study of economics"\n},\n"correct": "b"\n},\n"2": {\n"mcq": "Which theme in biology explains the unity and diversity of life?",\n"options": {\n"a": "Evolution",\n"b": "Chemistry",\n"c": "Physics",\n"d": "Genetics"\n},\n"correct": "a"\n},\n"3": {\n"mcq": "What is the importance of energy processing in organisms?",\n"options": {\n"a": "It allows organisms to communicate",\n"b": "It allows organisms to move, grow, and reproduce",\n"c": "It allows organisms to fly",\n"d": "It allows organisms to hibernate"\n},\n"correct": "b"\n},\n"4": {\n"mcq": "What is a key characteristic of all organisms mentioned in the text?",\n"options": {\n"a": "They are made up of rocks",\n"b": "They can\'t regulate their internal environments",\n"c": "They are able to regulate their own internal environments",\n"d": "They don\'t have genes"\n},\n"corre

In [25]:
quiz=json.loads(quiz)

In [26]:
quiz

{'1': {'mcq': 'What is biology?',
  'options': {'a': 'The study of rocks',
   'b': 'The scientific study of life',
   'c': 'The study of outer space',
   'd': 'The study of economics'},
  'correct': 'b'},
 '2': {'mcq': 'Which theme in biology explains the unity and diversity of life?',
  'options': {'a': 'Evolution',
   'b': 'Chemistry',
   'c': 'Physics',
   'd': 'Genetics'},
  'correct': 'a'},
 '3': {'mcq': 'What is the importance of energy processing in organisms?',
  'options': {'a': 'It allows organisms to communicate',
   'b': 'It allows organisms to move, grow, and reproduce',
   'c': 'It allows organisms to fly',
   'd': 'It allows organisms to hibernate'},
  'correct': 'b'},
 '4': {'mcq': 'What is a key characteristic of all organisms mentioned in the text?',
  'options': {'a': 'They are made up of rocks',
   'b': "They can't regulate their internal environments",
   'c': 'They are able to regulate their own internal environments',
   'd': "They don't have genes"},
  'correct'

In [27]:
quiz_data=[]

for key,value in quiz.items():
    mcq=value['mcq']
    options=" | ".join(
        [
             f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct_value=value['correct']
    quiz_data.append({
        "MCQ":mcq ,
        "CHOICES":options, 
        "CORRECT ANSWER":correct_value
                      })

In [28]:
quiz_data

[{'MCQ': 'What is biology?',
  'CHOICES': 'a: The study of rocks | b: The scientific study of life | c: The study of outer space | d: The study of economics',
  'CORRECT ANSWER': 'b'},
 {'MCQ': 'Which theme in biology explains the unity and diversity of life?',
  'CHOICES': 'a: Evolution | b: Chemistry | c: Physics | d: Genetics',
  'CORRECT ANSWER': 'a'},
 {'MCQ': 'What is the importance of energy processing in organisms?',
  'CHOICES': 'a: It allows organisms to communicate | b: It allows organisms to move, grow, and reproduce | c: It allows organisms to fly | d: It allows organisms to hibernate',
  'CORRECT ANSWER': 'b'},
 {'MCQ': 'What is a key characteristic of all organisms mentioned in the text?',
  'CHOICES': "a: They are made up of rocks | b: They can't regulate their internal environments | c: They are able to regulate their own internal environments | d: They don't have genes",
  'CORRECT ANSWER': 'c'},
 {'MCQ': 'What is the diversity of life on Earth attributed to?',
  'CHO

In [29]:
import pandas as pd
quiz=pd.DataFrame(quiz_data)

In [30]:
quiz

,MCQ,CHOICES,CORRECT ANSWER
0,What is biology?,a: The study of rocks | b: The scientific stud...,b
1,Which theme in biology explains the unity and ...,a: Evolution | b: Chemistry | c: Physics | d: ...,a
2,What is the importance of energy processing in...,a: It allows organisms to communicate | b: It ...,b
3,What is a key characteristic of all organisms ...,a: They are made up of rocks | b: They can't r...,c
4,What is the diversity of life on Earth attribu...,a: The number of planets in the solar system |...,b


In [31]:
quiz.to_csv('MachineLearning.csv',index=False)